**<h1>INSTALL NECESSARY LIBRARIES</h1>**

In [2]:
pip install -U fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 856.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227139 sha256=b87bae46953da44b3772f0503186f3c6264dbe22ad01a5adf008ebd459e0a96a
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [4]:
pip install -U num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=90abf63e30944dee9f0476482831ca4b9c848e325e530485721e2e81dd69953f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


**<h1>IMPORT LIBRARIES</h1>**

In [5]:
import re
import fasttext
import pandas as pd
from datasets import load_dataset
from num2words import num2words

**<h1>READ DATASET AND SAVING CORPUS</h1>**

In [6]:
corpus = load_dataset('ajaykarthick/imdb-movie-reviews')
corpus

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 10000
    })
})

In [7]:
train_dataset = corpus['train']
train = pd.DataFrame(train_dataset)
train[['review']]

,review
0,"Ms Aparna Sen, the maker of Mr & Mrs Iyer, dir..."
1,"I have seen this film only once, on TV, and it..."
2,I was only fourteen when I first saw the Alien...
3,This marvelous short will hit home with everyo...
4,If you are 10 years old and never seen a movie...
...,...
39995,"Why is it that any film about Cleopatra, the l..."
39996,In the area of movies based off of screenplays...
39997,A recent survey of children in the UK re-enfor...
39998,Absolute must see documentary for anyone inter...


In [8]:
test_dataset = corpus['test']
test = pd.DataFrame(test_dataset)
test[['review']]

,review
0,Essentially plotless action film has two good ...
1,I grew up on this movie and I can remember whe...
2,This was the best movie I've ever seen about B...
3,This film is about a bunch of misfits who are ...
4,"Bad plot, bad acting, bad direction.<br /><br ..."
...,...
9995,A beloved and devoted priest from a small town...
9996,"I've now seen this film twice, and I must say ..."
9997,I absolutely LOVED this movie! Since I don't w...
9998,Eddie Monroe is Hooooot. He is a great actor a...


In [9]:
old_index = test.index

In [10]:
index_mapping = {old_index: old_index + 40000 for old_index in range(10000)}

In [11]:
test_reindexed = test.rename(index=index_mapping)

In [12]:
combined_dataset = pd.concat([train, test_reindexed])
combined_dataset

,review,label
0,"Ms Aparna Sen, the maker of Mr & Mrs Iyer, dir...",0
1,"I have seen this film only once, on TV, and it...",0
2,I was only fourteen when I first saw the Alien...,1
3,This marvelous short will hit home with everyo...,0
4,If you are 10 years old and never seen a movie...,1
...,...,...
49995,A beloved and devoted priest from a small town...,0
49996,"I've now seen this film twice, and I must say ...",1
49997,I absolutely LOVED this movie! Since I don't w...,0
49998,Eddie Monroe is Hooooot. He is a great actor a...,0


**<h1>CLEAN CORPUS</h1>**

In [13]:
def convert_number_to_words(text):
    words = []
    for word in text.split():
        if word.isdigit():
            words.append(num2words(int(word)))
        else:
            words.append(word)
    return ' '.join(words)

In [31]:
def cleaning(text):
  text = re.sub(r'<br\s?/?>', ' ', text) #remove line-break tags
  text = text.replace('&', 'and') # replace '&' with 'and'
  text = re.sub(r'\([^)]*\)', '', text) # replace parentheses
  text = re.sub(r"(?<!\w)'|'(?!\w)|[^a-zA-Z\s']", ' ', text) # remove all non=letter characters except single quotes that is part of a word
  text = re.sub(r'[.,!?]', ' ', text) # remove punctuations
  text = re.sub(r' +', ' ', text) # remove double space
  return text.strip().lower() # return all the words in lowercase

In [15]:
combined_dataset['review'] = combined_dataset['review'].apply(convert_number_to_words)

In [32]:
combined_dataset['review'] = combined_dataset['review'].apply(cleaning)

In [33]:
combined_dataset['review'].loc[0]

"ms aparna sen the maker of mr and mrs iyer directs this movie about a young girl's struggle to cope with her debilitating condition meethi has been an aloof kid ever since childhood and has shown signs of delusion no one knows why the dormant tendency however slips out of control when the job assignment takes her to neighboring bihar where she's raped by some political goons the resulting trauma also leads to episodes of manic depressive psychosis in addition to her schizophrenia she careens out of control over the years progressively getting worse and sinking deeper into her private world the juxtaposition of an unsettled elder sister and how her domineering ways make an already bad situation worse is indicative of what a fine line there is between abnormal and seemingly normal ms sen also makes an excellent commentary on the social alienation of such individuals social rehab is standard therapy along with all the deadly mind altering drugs but what about the poor and the destitute w

In [34]:
combined_dataset.to_csv('corpus.txt', columns=['review'], header=None, index=False)

**<h1>EMBEDDING WORDS</h1>**

In [35]:
model = fasttext.train_unsupervised('corpus.txt')

In [36]:
example = 'a fantasy adventure that fuses greek mythology to contemporary american places and values anyone around fifteen give or take a couple of years will thrill to the visual spectacle'

In [37]:
model.get_word_vector(example)

array([ 0.0311037 ,  0.03019407, -0.01089972, -0.0640444 ,  0.08152802,
       -0.01300095, -0.1273184 ,  0.04413427,  0.06330308,  0.0492626 ,
       -0.02754412,  0.01716277, -0.0485076 , -0.09071079,  0.10256448,
        0.09364121, -0.0344948 , -0.05125597,  0.0029415 , -0.04960882,
        0.020182  ,  0.01108337, -0.08915768, -0.03453002, -0.00848341,
       -0.01996987,  0.16583474, -0.00026714, -0.16926865,  0.0726763 ,
        0.0111618 ,  0.04168399, -0.17456481, -0.00597573, -0.05850693,
        0.01560966, -0.05623416, -0.09809875,  0.02040096,  0.09918267,
       -0.10504108, -0.05256776,  0.03995062, -0.004407  ,  0.04316057,
       -0.04793288,  0.153643  , -0.01910378,  0.00309787,  0.04589375,
        0.02596209, -0.06924402, -0.11084215, -0.03324487, -0.04554943,
       -0.055763  , -0.1187463 , -0.04843805,  0.08632246,  0.0575395 ,
       -0.05113632,  0.06478859,  0.03223842, -0.02678363, -0.18106502,
       -0.07625299, -0.17507857,  0.12398227, -0.0346706 , -0.02

In [38]:
dataset = [
    'uma thurman as medusa the gorgon with a coiffure of writhing snakes and stoneinducing hypnotic gaze is one of the highlights of this bewitching fantasy',
    'with a topnotch cast and dazzling special effects this will tide the teens over until the next harry potter instalment',
    'whether audiences will get behind the lightning thief is hard to predict overall its an entertaining introduction to a promising new world  but will the consuming shadow of potter be too big to break free of'
    ]

In [39]:
word_embeddings = [model.get_word_vector(word) for word in dataset]

In [40]:
word_embeddings[0]

array([-0.01227314,  0.02922275, -0.13004333, -0.08487002,  0.05081871,
       -0.05225871, -0.10845922,  0.0799403 ,  0.06378125,  0.08350796,
       -0.00776916, -0.05654016, -0.01740745,  0.01666541,  0.06041095,
       -0.00346929, -0.02637514, -0.01576164, -0.02977703, -0.0463409 ,
        0.05588118, -0.07408123, -0.07937741, -0.04840459, -0.05076971,
       -0.02635338,  0.09934992, -0.06145253, -0.14666022,  0.07987126,
        0.05236699,  0.03145798, -0.10505041, -0.00872199, -0.04364416,
        0.05873332, -0.01572749, -0.05688619, -0.03065411,  0.12470208,
       -0.04418175,  0.05133809,  0.05650416,  0.04849526, -0.04554445,
        0.00933753,  0.01987064,  0.05854888, -0.03899737,  0.09514502,
        0.02586043, -0.08389938, -0.09174877, -0.08090519, -0.06261272,
       -0.0653913 , -0.10308675, -0.04326242,  0.12912859,  0.01804825,
        0.00557107,  0.02655249,  0.07506245, -0.05646586, -0.07651138,
       -0.09442966, -0.15934226,  0.03540765,  0.05877106, -0.03